In [1]:
import pandas as pd
import numpy as np

In [2]:
bv = pd.read_csv("../data/external/bill_version.csv", sep=";", encoding="latin1", parse_dates=True).rename(columns={"id": "version_id"})
bills =  pd.read_csv("../data/external/bill.csv", sep=";", encoding="latin1", parse_dates=True).rename(columns={"id": "bill_id"})

In [3]:
partisan_lean = pd.read_csv("../data/derived/partisan_lean.csv", sep=",", encoding="latin1", parse_dates=True)
derived_features = pd.read_csv("../data/derived/basic_version_features.csv", sep=",", encoding="latin1", parse_dates=True).rename(columns={'id': 'version_id'})

In [4]:
df = bv[['version_id', 'bill_id']].merge(bills[['bill_id', 'chamber_id', 'session_id', 'signed']], on = 'bill_id')

In [6]:
df = df.fillna(0)
df.loc[df.signed != 0, 'signed'] = 1
df2 = df.merge(partisan_lean[['abbr', 'session_id', 'chamber_id', 'partisan_lean']], on = ['session_id', 'chamber_id'])

In [13]:
df_ml = df2.merge(derived_features, on = 'version_id')
df_ml_X = df_ml[['session_id', 'partisan_lean', 'text_length']]
df_ml_Y = df_ml[['signed']]

In [10]:
from sklearn.model_selection import train_test_split

In [15]:
[X_train, X_test, Y_train, Y_test] = train_test_split(df_ml_X, df_ml_Y, test_size = 0.1)

In [16]:
X_train.shape

(446632, 3)

In [17]:
import xgboost as xgb